In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from xgboost import XGBRegressor

train.corr(method='pearson')
correlation_heatmap(train, "")



y = train.dropna().close
X = train.dropna().drop(['close'], axis=1)
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

xgb = XGBRegressor()
xgb.fit(X_train_scaled, y_train);


df = pd.DataFrame({'value' : abs(xgb.feature_importances_), 'name' : X_train.columns.tolist()})
features = df.loc[df.value!=0.0].sort_values('value', ascending=False).reset_index(drop=True)
features

In [ ]:
%%time
from dask_ml.xgboost import XGBRegressor
from dask.distributed import Client
client = Client()
#from dask_ml.model_selection import train_test_split

# ['tags', 'correct_answer',  'bundle_id',  'user_id',  'content_id', 'task_container_id']
var = ['timestamp', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'part_5',
                                                     'part_1', 'part_2', 'part_3', 'part_4', 'part_6', 'part_7']

xgb_train =  train[var]
xgb_label = train[dep_var]

xgb = XGBRegressor()
xgb.fit(xgb_train, xgb_label)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/riiideducation/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/riiideducation/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/riiideducation/lib/python3.8/site-packages/dask_xgboost/tracker.py", line 358, in run
    self.accept_slaves(nslave)
  File "/home/ubuntu/anaconda3/envs/riiideducation/lib/python3.8/site-packages/dask_xgboost/tracker.py", line 296, in accept_slaves
    s = SlaveEntry(fd, s_addr)
  File "/home/ubuntu/anaconda3/envs/riiideducation/lib/python3.8/site-packages/dask_xgboost/tracker.py", line 85, in __init__
    magic = slave.recvint()
  File "/home/ubuntu/anac

In [1]:
import pandas as pd, warnings
import dask.dataframe as dd
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
%%time
train = dd.read_csv('data/train.csv', dtype={'row_id': 'int64',
                              'timestamp': 'int64',
                              'user_id': 'int32',
                              'content_id': 'str',
                              'content_type_id': 'int8',
                              'task_container_id': 'int16',
                              'user_answer': 'int8',
                              'answered_correctly': 'int8',
                              'prior_question_elapsed_time': 'float32', 
                              'prior_question_had_explanation': 'boolean',
                             })
del train['row_id']

def Normalizer(dd, const_names):
    """
    Normalized constant columns in a dask dataframe.
    """
    for name in const_names:
        if name in ['timestamp', 'prior_question_elapsed_time']:
            dd[name] = (dd[name] - dd[name].min()) / (dd[name].max() - dd[name].min())
        else:
            mean, std = dd[name].mean().compute(), dd[name].std().compute()
            dd[name] = (dd[name] - mean) / (1e-7 + std)
    return dd


def description(df):
    """
    Return basic statistics.
    """
    summary = pd.DataFrame(df.dtypes, columns=['dtypes']).reset_index()
    summary = summary.rename(columns={'index':'name'})
    summary['Missing'] = df.isnull().sum().values
    summary['nunique'] = df.nunique().values
    summary['unique'] = df.apply(lambda x: list(x.unique()), axis=0).reset_index()[0]
    return summary

test, lectures, questions = dd.read_csv('data/example_test.csv'), dd.read_csv('data/lectures.csv'),\
                            dd.read_csv('data/questions.csv', dtype={'questions_id':'str'})
print('Train shape: {0}\nTest shape: {1}'.format(train.shape[0].compute(), test.shape[0].compute()))

Train shape: 101230332
Test shape: 104
CPU times: user 2min 50s, sys: 15.5 s, total: 3min 6s
Wall time: 2min 18s


In [3]:
%%time
description(train.compute())

CPU times: user 3min 41s, sys: 25.9 s, total: 4min 7s
Wall time: 3min 16s


,name,dtypes,Missing,nunique,unique
0,timestamp,int64,0,72821015,"[0, 56943, 118363, 131167, 137965, 157063, 176..."
1,user_id,int32,0,393656,"[115, 124, 2746, 5382, 8623, 8701, 12741, 1313..."
2,content_id,int16,0,13782,"[5692, 5716, 128, 7860, 7922, 156, 51, 50, 789..."
3,content_type_id,int8,0,2,"[0, 1]"
4,task_container_id,int16,0,10000,"[1, 2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,user_answer,int8,0,5,"[3, 2, 0, 1, -1]"
6,answered_correctly,int8,0,3,"[1, 0, -1]"
7,prior_question_elapsed_time,float32,2351538,3258,"[nan, 37000.0, 55000.0, 19000.0, 11000.0, 5000..."
8,prior_question_had_explanation,boolean,392506,2,"[<NA>, False, True]"


In [3]:
%%time
train = train.map_partitions(lambda x: x[~x.index.isin(x.loc[x.user_answer==-1].index)]).reset_index(drop=True)
questions = questions.rename(columns={'question_id':'content_id'}).set_index('content_id')
lectures = lectures.rename(columns={'lecture_id':'content_id'}).set_index('content_id')
train.content_id, questions.index, lectures.index = train.content_id.astype('str'), questions.index.astype('str'),\
                                                    lectures.index.astype('str')


train = train.join(lectures, on='content_id')
train = train.join(questions, on='content_id', rsuffix='_Qu')
train = train.drop(['answered_correctly', 'tag', 'part', 'type_of', 'content_type_id', 'correct_answer'], axis=1)

train.head(5)

CPU times: user 3.14 s, sys: 369 ms, total: 3.51 s
Wall time: 3.47 s


,timestamp,user_id,content_id,task_container_id,user_answer,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,part_Qu,tags
0,0,115,5692,1,3,NaN,<NA>,5692,5,151
1,56943,115,5716,2,2,37000.0,False,5716,5,168
2,118363,115,128,0,0,55000.0,False,128,1,131 149 92
3,131167,115,7860,3,0,19000.0,False,7860,1,131 104 81
4,137965,115,7922,4,1,11000.0,False,7922,1,131 149 92


In [4]:
dep_var = 'user_answer'
cat_names = ['part_Qu', 'bundle_id', 'content_id', 'task_container_id', 'tags']
const_names = ['timestamp', 'prior_question_elapsed_time', 'prior_question_had_explanation']

In [5]:
train = train.categorize(columns = cat_names)
train = dd.get_dummies(train, columns = ['part_Qu'], prefix = ['part'])
train = train.dropna()
train = Normalizer(train, const_names[:2])

In [13]:
%%time
description(train.compute())

CPU times: user 6min 59s, sys: 41.2 s, total: 7min 40s
Wall time: 6min 13s


,name,dtypes,Missing,nunique,unique
0,timestamp,float64,0,70897330,"[6.513296784852509e-07, 1.3538685129787637e-06..."
1,user_id,int32,0,393569,"[115, 124, 2746, 5382, 8623, 8701, 12741, 1313..."
2,content_id,category,0,13522,"[5716, 128, 7860, 7922, 156, 51, 50, 7896, 786..."
3,task_container_id,category,0,10000,"[2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,user_answer,int8,0,4,"[2, 0, 1, 3]"
5,prior_question_elapsed_time,float32,0,3258,"[0.123333335, 0.18333334, 0.06333333, 0.036666..."
6,prior_question_had_explanation,boolean,0,2,"[False, True]"
7,bundle_id,category,0,9764,"[5716, 128, 7860, 7922, 156, 51, 50, 7896, 786..."
8,correct_answer,int64,0,4,"[2, 0, 1, 3]"
9,tags,category,0,1519,"[168, 131 149 92, 131 104 81, 131 101 162 38, ..."


In [ ]:
train.shape[0].compute()

In [19]:
lectures.head()

,lecture_id,tag,part,type_of
0,89,24584,5,concept
1,100,22243,1,concept
2,185,7035,6,concept
3,192,31458,5,solving question
4,317,19653,5,solving question


In [14]:
description(lectures.compute())

,name,dtypes,Missing,nunique,unique
0,lecture_id,int64,0,418,"[89, 100, 185, 192, 317, 335, 484, 641, 761, 8..."
1,tag,int64,0,151,"[24584, 22243, 7035, 31458, 19653, 32051, 8462..."
2,part,int64,0,7,"[5, 1, 6, 2, 7, 3, 4]"
3,type_of,object,0,4,"[concept, solving question, intention, starter]"


In [20]:
questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [28]:
description(questions.compute())

,name,dtypes,Missing,nunique,unique
0,question_id,int64,0,13523,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,bundle_id,int64,0,9765,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,correct_answer,int64,0,4,"[0, 1, 3, 2]"
3,part,int64,0,7,"[1, 2, 3, 4, 5, 6, 7]"
4,tags,object,1,1519,"[51 131 162 38, 131 36 81, 131 101 162 92, 131..."


In [8]:
#%%time
#train.map_partitions(lambda x: x['user_answer'] == x['answered_correctly']).compute()

#They are group's info in the test data but not in the train set. 
#--> Not used columns ['group_num', 'prior_group_answers_correct', 'prior_group_responses']
list(lectures.columns), list(questions.columns), list(test.columns)

(['lecture_id', 'tag', 'part', 'type_of'],
 ['question_id', 'bundle_id', 'correct_answer', 'part', 'tags'],
 ['row_id',
  'group_num',
  'timestamp',
  'user_id',
  'content_id',
  'content_type_id',
  'task_container_id',
  'prior_question_elapsed_time',
  'prior_question_had_explanation',
  'prior_group_answers_correct',
  'prior_group_responses'])

Files: train.csv

**row_id**: (int64) ID code for the row.

**timestamp**: (int64) the time between this user interaction and the first event from that user.

**user_id**: (int32) ID code for the user.

**content_id**: (int16) ID code for the user interaction

**content_type_id**: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

**task_container_id**: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id. Monotonically increasing for each user.

**user_answer**: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

**answered_correctly**: (int8) if the user responded correctly. Read -1 as null, for lectures.

**prior_question_elapsed_time**: (float32) How long it took a user to answer their previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Note that the time is the total time a user took to solve all the questions in the previous bundle.

**prior_question_had_explanation**: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

**questions.csv: metadata for the questions posed to users.**

**question_id**: foreign key for the train/test content_id column, when the content type is question (0).

**bundle_id**: code for which questions are served together.

**correct_answer**: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

**part**: top level category code for the question.

**tags**: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

**lectures.csv: metadata for the lectures watched by users as they progress in their education.**

**lecture_id**: foreign key for the train/test content_id column, when the content type is lecture (1).

**part**: top level category code for the lecture.

**tag**: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

**type_of**: brief description of the core purpose of the lecture

In [30]:
train['tags'].head()

1               168
2        131 149 92
3        131 104 81
4        131 149 92
5    131 101 162 38
Name: tags, dtype: category
Categories (1519, object): ['151', '168', '131 149 92', '131 104 81', ..., '98 76 160 135 162', '113 12 29', '98 84 76 135', '136 12 162 29']

In [32]:
train['bundle_id'].head()

1    5716
2     128
3    7860
4    7922
5     156
Name: bundle_id, dtype: category
Categories (9765, int64): [5692, 5716, 128, 7860, ..., 7546, 10033, 5823, 10005]